In [ ]:
#%pip install torch==1.10.0 torchvision==0.11.1 torchaudio==0.10.0 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [5]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

wave3vecfeatures.py
This program does the following:
    1. Extract features from each audio files for each transformer layer in wave2 vec2
    2. Please note the pattern is same as that librosa feature extraction
    
"""
print('Now doing Hubert')
#import fairseq
import torch
#from fairseq.models.wav2vec import Wav2Vec2Model,Wav2VecModel
import librosa
import numpy as np
import os
#import IPython
import matplotlib
import matplotlib.pyplot as plt
#import requests

import torchaudio
#from torchvision import datasets, transforms, models
#import librosa, librosa.display
#import matplotlib.pyplot as plt
import pickle
#import os

import numpy as np




#recrd = "139_39117_1587600000.wav"
DATASET_PATH = "../samples/voice_samples"

import glob
filenames= glob.glob(DATASET_PATH+"/*.wav" )
filenames = [os.path.basename(x) for x in filenames]


from joblib import Parallel, delayed




Now doing Hubert


In [13]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [3]:
DATASET_PATH = "../samples/voice_samples"
device = torch.device("cpu")
bundle = torchaudio.pipelines.WAV2VEC2_BASE
model = bundle.get_model().to(device)



Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

In [6]:
pkl= "../data/w2v2L8"
def process(recrd):    
    file = DATASET_PATH +"/"+ recrd
    flo = recrd.split('.')[0]

    print("model downloaded")   
    from os.path import exists
    path_to_file= pkl+'/'+flo+'.pickle'
    if exists(path_to_file)==False:
        waveform, sample_rate = torchaudio.load(file)
        waveform = waveform.to(device)
        #print(recrd)
        if sample_rate != bundle.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
        with torch.inference_mode():
            features, _ = model.extract_features(waveform)        
        with open(path_to_file, 'wb') as handle:
            pickle.dump(features[8], handle, protocol=pickle.HIGHEST_PROTOCOL)
        features= None
        

results = Parallel(n_jobs=-1)(delayed(process)(recrd) for recrd in filenames)

/home/ubuntu/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
